<a href="https://colab.research.google.com/github/Tigropoil/SAE_S6/blob/Arthur/text_classification_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Installing Dependencies for Binary Text Classification with BERT**

This code block installs the necessary Python libraries for building a text multi-classification script using a BERT model in a Google Colab notebook. The libraries at play include PyTorch (for deep learning), Transformers (for accessing pre-trained models like BERT), NumPy (for efficient array manipulation), Pandas (for data manipulation and analysis), Scikit-learn (for machine learning algorithms), Datasets (for managing and loading datasets), and tqdm (for displaying progress bars).

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install torch transformers numpy pandas scikit-learn
!pip install datasets
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

# 📚 **Dataset Preparation & Tokenization** 📚

In this section, we load the IMDB reviews dataset, preprocess the data, and tokenize the text using BERT's tokenizer. Shuffling and splitting the dataset into training and validation sets ensure a better and unbiased model evaluation. BERT tokenizer is utilized to convert the raw text into a format understandable by the pre-trained BERT model. This is an essential step before feeding your data into the model for training.

In [14]:
data_url = '/content/drive/MyDrive/SAE S6/data_fusion_little.csv'

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer
import torch
from tqdm import tqdm

data = pd.read_csv(data_url)

In [16]:
# display data

#data.head()

data.columns

Index(['Titre', 'Description', 'Auteurs', 'Image', 'Lien Google', 'Editeur',
       'Date publication', 'infoLink', 'Genre', 'Nb scores', 'Id', 'Prix',
       'User_id', 'Nom lecteur', 'revue/utilité', 'revue/score', 'revue/heure',
       'revue/résumé', 'revue/texte'],
      dtype='object')

In [17]:
data['revue/texte']

,revue/texte
0,This is only for Julie Strain fans. It's a col...
1,I don't care much for Dr. Seuss but after read...
2,"If people become the books they read and if ""t..."
3,"I just finished the book, &quot;Wonderful Wors..."
4,Many small churches feel like they can not hav...
...,...
359935,"On the front cover of the novel MAMAW, there i..."
359936,After reading 50 pages and restarting every so...
359937,What is wrong or good with this book. What are...
359938,Having a teacher that loved this book didn't h...


In [18]:
data = data.drop(['Image', 'Lien Google', 'infoLink', 'Id', 'revue/score'], axis=1)

# Split the dataset into training and validation sets
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Tokenize the data using BERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
max_seq_len = 128

train_input_ids, train_attention_masks, train_labels = tokenize_data(train_data, tokenizer, max_seq_len)
val_input_ids, val_attention_masks, val_labels = tokenize_data(val_data, tokenizer, max_seq_len)

  0%|          | 0/287952 [00:00<?, ?it/s]


KeyError: 'label'

# 🔀 **Batch Processing with DataLoader** 🔀

After tokenizing the data, this section focuses on creating DataLoader objects for the training and validation sets. DataLoader helps with efficiently processing the data in batches, enabling better resource management during model training and evaluation. This step makes your dataset ready for the subsequent model training and evaluation stages.

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 16

# Create DataLoader for the training set
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

# Create DataLoader for the validation set
val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels)
val_sampler = SequentialSampler(val_dataset)
val_dataloader = DataLoader(val_dataset, sampler=val_sampler, batch_size=batch_size)

# 🤖 **Loading the BERT Model for Classification Task** 🤖

Here, we configure and load a pre-trained BERT model for a specific classification task. This involves setting the model's output to the desired number of labels and disabling the output of unnecessary components like attention weights and hidden states. Moving the model to the GPU (if available) allows you to benefit from the accelerated training process.

In [ ]:
from transformers import DistilBertForSequenceClassification, AdamW, BertConfig

# Load the pre-trained BERT model
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,  # Use 2 labels for binary classification, adjust it for multi-class problems
    output_attentions=False,
    output_hidden_states=False,
)

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

# 🚂 **BERT Model Training** 🚂

This final part involves fine-tuning the BERT model on the provided dataset and evaluating its performance. We define functions to train and evaluate the model after every epoch and calculate loss and accuracy metrics during training and validation, respectively. Furthermore, components such as optimizer and scheduler are introduced for efficient model training to help improve the results on each step. This section helps you understand the overall process of training BERT for a classification task and assessing the model's performance.

In [ ]:
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, classification_report

num_epochs = 3
total_steps = len(train_dataloader) * num_epochs

optimizer = AdamW(model.parameters(), lr=2e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

def train_epoch(model, dataloader, optimizer, scheduler, device):
    model.train()
    total_loss = 0

    progress_bar = tqdm(dataloader, desc="Training", position=0, leave=True)
    for batch in progress_bar:
        input_ids, attention_masks, labels = [t.to(device) for t in batch]

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_masks, labels=labels)
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()

        progress_bar.set_description(f"Training - Loss: {loss.item():.4f}")

    return total_loss / len(dataloader)

def evaluate(model, dataloader, device):
    model.eval()
    total_eval_accuracy = 0

    progress_bar = tqdm(dataloader, desc="Evaluation", position=0, leave=True)
    for batch in progress_bar:
        input_ids, attention_masks, labels = [t.to(device) for t in batch]
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_masks)

        logits = outputs[0].detach().cpu().numpy()
        label_ids = labels.cpu().numpy()
        batch_accuracy = accuracy_score(label_ids, logits.argmax(axis=-1))
        total_eval_accuracy += batch_accuracy

        progress_bar.set_description(f"Evaluation - Batch Accuracy: {batch_accuracy:.4f}")

    return total_eval_accuracy / len(dataloader)

for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_dataloader, optimizer, scheduler, device)
    val_accuracy = evaluate(model, val_dataloader, device)

    print(f"\nEpoch {epoch + 1}/{num_epochs}")
    print(f"Loss: {train_loss:.4f} - Validation Accuracy: {val_accuracy:.4f}")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Evaluation - Batch Accuracy: 1.0000: 100%|██████████| 313/313 [00:18<00:00, 16.49it/s]



Epoch 1/3
Loss: 0.3574 - Validation Accuracy: 0.8778


Evaluation - Batch Accuracy: 1.0000: 100%|██████████| 313/313 [00:18<00:00, 16.49it/s]



Epoch 2/3
Loss: 0.2089 - Validation Accuracy: 0.8850


Evaluation - Batch Accuracy: 1.0000: 100%|██████████| 313/313 [00:18<00:00, 16.48it/s]


Epoch 3/3
Loss: 0.1167 - Validation Accuracy: 0.8880


# 📊 **Evaluating BERT Model Using Performance Metrics** 📊

Building on the previous model training and evaluation process, this section is dedicated to extracting the BERT model's predictions and comparing them with the true labels in the validation dataset. We define a function that gathers predictions and true labels, enabling the calculation of accuracy and a detailed classification report. This assessment step is crucial for identifying how well the model performs on unseen data.

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

def get_predictions(model, dataloader, device):
    model.eval()
    predictions, true_labels = [], []

    for batch in tqdm(dataloader, desc="Evaluating"):
        input_ids, attention_masks, labels = [t.to(device) for t in batch]
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_masks)

        logits = outputs[0].detach().cpu().numpy()
        label_ids = labels.cpu().numpy()
        predictions.extend(logits.argmax(axis=-1))
        true_labels.extend(label_ids)

    return np.array(predictions), np.array(true_labels)

predictions, true_labels = get_predictions(model, val_dataloader, device)
accuracy = accuracy_score(true_labels, predictions)
report = classification_report(true_labels, predictions, digits=4)

print(f"Validation Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(report)

Evaluating: 100%|██████████| 313/313 [00:19<00:00, 15.66it/s]

Validation Accuracy: 0.8878
Classification Report:
              precision    recall  f1-score   support

           0     0.8916    0.8819    0.8867      2489
           1     0.8842    0.8937    0.8889      2511

    accuracy                         0.8878      5000
   macro avg     0.8879    0.8878    0.8878      5000
weighted avg     0.8878    0.8878    0.8878      5000



# 💾 **Save Model**

In [ ]:
output_dir = "./model/"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('./model/tokenizer_config.json',
 './model/special_tokens_map.json',
 './model/vocab.txt',
 './model/added_tokens.json')

# 🔍 **Classifying a Movie Review with the BERT Model** 🔍

In this section, we present the code to classify a sample movie review using our trained BERT model. By providing a text string representing a movie review, the code tokenizes the text and feeds it into the model to obtain the predicted label index and class. This is a practical application of the BERT model's capabilities and demonstrates how the model can be used to classify new, unseen data. You can test the model's performance on any desired movie review text by simply replacing the `review` string in the code.

In [ ]:
def predict_sentiment(review, model, tokenizer, device):
    model.eval()

    # Tokenize the input text
    encoded = tokenizer.encode_plus(
        review,
        add_special_tokens=True,  # Add [CLS] and [SEP]
        max_length=128,  # Set maximum sequence length
        padding="max_length",  # Pad the sequence if it is shorter than max_seq_len
        truncation=True,  # Truncate the sequence if it is longer than max_seq_len
        return_attention_mask=True,  # Return the attention mask
    )

    input_id = torch.tensor([encoded["input_ids"]]).to(device)
    attention_mask = torch.tensor([encoded["attention_mask"]]).to(device)

    with torch.no_grad():
        outputs = model(input_id, attention_mask=attention_mask)

    logits = outputs[0].detach().cpu().numpy()
    sentiment = logits.argmax(axis=-1)[0]

    return sentiment

In [ ]:
review = "I really enjoyed this movie. The actors were great and the plot was engaging."
predicted_sentiment = predict_sentiment(review, model, tokenizer, device)

if predicted_sentiment == 0:
    print("Negative sentiment")
else:
    print("Positive sentiment")

Positive sentiment
